In [20]:
TENSORFLOW = 1
GLUON = 1

import os  
if GLUON:
    os.environ['CUDA_VISIBLE_DEVICES']='1'

if TENSORFLOW:
    import tensorflow as tf

    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.set_visible_devices(gpus[0], 'GPU')
            tf.config.set_logical_device_configuration(
                gpus[0],
                [tf.config.LogicalDeviceConfiguration(memory_limit=24000)])
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)


1 Physical GPUs, 1 Logical GPUs


In [21]:
import warnings, pandas as pd, requests, mlflow, sys, os, logging, numpy as np
from automlwrapper import AutoMLWrapper, SedarDataLoader
from sedarapi import SedarAPI
from sklearn.model_selection import train_test_split
from autogluon.multimodal.utils.object_detection import from_coco
from PIL import Image

logging.getLogger('automlwrapper').setLevel(logging.ERROR)

np.random.seed(1313)

TIME_LIMIT = 60 * 60

In [22]:
USE_MLFLOW = False

MLFLOW_URI = 'http://192.168.220.107:6798'
MLFLOW_EXPID = '1'
if USE_MLFLOW:
    mlflow.set_tracking_uri(MLFLOW_URI) 
    mlflow.set_experiment(experiment_id=MLFLOW_EXPID)

In [23]:
SEDAR_URI = 'http://192.168.220.107:5000'

SEDAR = SedarAPI(SEDAR_URI)
DataLoader = SedarDataLoader(SEDAR)

In [24]:
train_location = './tmp/data/gitterschnitte_segmentation_train'
test_location = './tmp/data/gitterschnitte_segmentation_test'
val_location = './tmp/data/gitterschnitte_segmentation_val'

## download data to the above locations

In [6]:
!curl --create-dirs -O --output-dir \
./tmp/data/gitterschnitte_segmentation_train \
https://gitlab.com/mibbels/automlwrapperdata/-/raw/main/image-segmentation/gitterschnitte_segmentation_train.zip 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7173k  100 7173k    0     0  16.8M      0 --:--:-- --:--:-- --:--:-- 16.8M


In [7]:
!curl --create-dirs -O --output-dir \
./tmp/data/gitterschnitte_segmentation_test \
https://gitlab.com/mibbels/automlwrapperdata/-/raw/main/image-segmentation/gitterschnitte_segmentation_test.zip 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  345k  100  345k    0     0   977k      0 --:--:-- --:--:-- --:--:--  979k


In [8]:
!curl --create-dirs -O --output-dir \
./tmp/data/gitterschnitte_segmentation_val \
https://gitlab.com/mibbels/automlwrapperdata/-/raw/main/image-segmentation/gitterschnitte_segmentation_val.zip 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1337k  100 1337k    0     0  3503k      0 --:--:-- --:--:-- --:--:-- 3502k


## Read the segmentation dataset from files
#### Same code utilizing the SEDAR API:
```python
DataLoader.query_data('<workspace id>',
                      '<dataset id>',
                      file_save_location = test_location)
```

In [25]:
train = DataLoader.segmentation_as_data_frame(train_location, train_location + '/unzip')

test = DataLoader.segmentation_as_data_frame(test_location, test_location + '/unzip')

val = DataLoader.segmentation_as_data_frame(val_location, val_location + '/unzip')

In [10]:
train.describe()

,image,label
count,174,174
unique,174,174
top,./tmp/data/gitterschnitte_segmentation_train/u...,./tmp/data/gitterschnitte_segmentation_train/u...
freq,1,1


In [11]:
test.describe()

,image,label
count,8,8
unique,8,8
top,./tmp/data/gitterschnitte_segmentation_test/un...,./tmp/data/gitterschnitte_segmentation_test/un...
freq,1,1


In [12]:
val.describe()

,image,label
count,35,35
unique,35,35
top,./tmp/data/gitterschnitte_segmentation_val/unz...,./tmp/data/gitterschnitte_segmentation_val/unz...
freq,1,1


# Optimizing the smallest version of the SAM-Network using AutoGluon

In [36]:
def wrapper(train_data, val_data, eval_metric):
    
    wrapper = AutoMLWrapper('autogluon')
    wrapper.AllowExtraHyperparameters({'constructor': ['hyperparameters']})
    wrapper.Train(
        train_data=train_data,
        validation_data=val_data,
        target_column='label',
        task_type='segmentation',
        data_type='image',
        problem_type='segmentation',
        hyperparameters={'eval_metric':eval_metric,
                         'validation_metric':eval_metric,
                         'time_limit':TIME_LIMIT,
                         'verbosity':0, 
                         'hyperparameters':{
                             "model.sam.checkpoint_name": "facebook/sam-vit-base",
                         }
                        }
    )
    
    if USE_MLFLOW:
         additional_tags = {
         "workspace":"DefaultWorkspace,13b4787c3e454649aa05a4cd680edc37",
         "notebook": "glass classifiaction PB,854c94fd-1898-4f03-b0c6-b60b5571b102",
         "is_public":"true",
         "datasets":"glass_schema|986f2e837ca44f3e8c0ee7d2dc0c4287",
         "username":"mibbels",
         }
         n = 3
         wrapper.MlflowUploadTopN(n, additional_tags)

    return wrapper



In [37]:
%%time
w = wrapper(train, val, 'iou')

AutoMLWrapper: Warning - num_classes not found in params. Setting num_classes to 1 in order to use binary_iou as eval_metric.


Global seed set to 0
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name              | Type                       | Params
-----------------------------------------------------------------
0 | model             | SAMForSemanticSegmentation | 93.4 M
1 | validation_metric | Binary_IoU                 | 0     
2 | loss_func         | StructureLoss              | 0     
-----------------------------------------------------------------
3.6 M     Trainable params
89.8 M    Non-trainable params
93.4 M    Total params
373.703   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/automl/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Epoch 10, global step 484: 'val_iou' was not in top 3


Validation: 0it [00:00, ?it/s]

Epoch 11, global step 528: 'val_iou' was not in top 3


CPU times: user 12min 39s, sys: 16.4 s, total: 12min 56s
Wall time: 8min 43s


In [38]:
%%time
test_iou = w.Evaluate(test, target_column='label', metrics=['iou'])

print(test_iou)

Predicting: 0it [00:00, ?it/s]

{'iou': 0.818583607673645}
CPU times: user 6.78 s, sys: 1.07 s, total: 7.85 s
Wall time: 1.82 s


--------------
# Evaluating the LLMs capabilities when creating segmentation code

In [ ]:
train = DataLoader.segmentation_as_data_frame(train_location, train_location + '/unzip')

test = DataLoader.segmentation_as_data_frame(test_location, test_location + '/unzip')

val = DataLoader.segmentation_as_data_frame(val_location, val_location + '/unzip')

### **Trial 1**: The prompt currently used within the SEDAR System

In [39]:
print(f'''                                         
Generate a Python code snippet for an Image Segmentation pipeline using the libraries Tensorflow or Pytorch and other appropriate libraries, optimized for GPU usage. 
The code should handle a dataframe called gitter_trainv1 with columns 'image' and 'label'. 'image' contains the path to the image file and 'label' contains the path to the mask file.
The dataset contains 348 images. 
All images are 256px in height, 256px in width, in GRAY mode, and saved as png.

Focus on a simple yet effective neural network architecture suitable for beginners. You may use a pre-trained model and fine-tune it on the dataset.
Include necessary imports, dataset loading and preprocessing steps, model architecture, and training loop. The code should be ready to run in a Python interpreter without modifications.
Annotate the code with comments to explain each step and give hints for continued development of that pipeline.
''')

                                         
Generate a Python code snippet for an Image Segmentation pipeline using the libraries Tensorflow or Pytorch and other appropriate libraries, optimized for GPU usage. 
The code should handle a dataframe called gitter_trainv1 with columns 'image' and 'label'. 'image' contains the path to the image file and 'label' contains the path to the mask file.
The dataset contains 348 images. 
All images are 256px in height, 256px in width, in GRAY mode, and saved as png.

Focus on a simple yet effective neural network architecture suitable for beginners. You may use a pre-trained model and fine-tune it on the dataset.
Include necessary imports, dataset loading and preprocessing steps, model architecture, and training loop. The code should be ready to run in a Python interpreter without modifications.
Annotate the code with comments to explain each step and give hints for continued development of that pipeline.



# When presented with that prompt, OpenOrca tries to create a classification network 
- the term 'label' as the name for the column with the mask image might confuse the LLM
- `tf.keras.preprocessing.image.load_filelist` does not exist

```python
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Load the dataset into a pandas dataframe
(df,) = tf.keras.preprocessing.image.load_filelist(
    gitter_trainv1['image'], 
    target_size=(256, 256), 
    batch_size=32, 
    class_mode='categorical', 
    subset='training'
)
#Reshape the data to have channels last
df = tf.keras.applications.mobilenet_v2.preprocess_input(df)
#Labels are one-hot encoded
df['label'] = keras.utils.to_categorical(df['label'], 10)
model = tf.keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)), layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(.5),
    layers.Dense(10) 
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)
#Train the model for a specified number of epochs
model.fit(df['image'], df['label'], batch_size=32, epochs=10)
```

# # **Trial 2**: The next step was to build the pipeline iteratively with the help of OpenOrca 
### Step 1: data conversion into arrays
Prompt 1:

I have a dataframe called gitter_trainv1 with columns 'image' and 'label'. 'image' contains the path to the image file and 'label' contains the path to the mask file. The dataset contains 174 images. All images are 256px in height, 256px in width, in GRAY mode, and saved as png. Write the python code that loads the images and masks into numpy arrays, so that I can use them to create a neural network for image segmentation in tensorflow.

In [26]:
import cv2
from PIL import Image
import numpy as np

# Loading Images and Masks from Dataframe
data_df = train
image_paths = list(data_df['image'])   # Extract image paths
mask_paths = list(data_df['label'])    # Extract mask paths

images = np.array([cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) for img_path in image_paths]) / 255.0
images = images.reshape(-1, 256, 256, 1)
    
labels = np.array([np.array(Image.open(mask_path).convert('L').resize((256, 256))) for mask_path in mask_paths]) / 255.0
labels = labels.reshape(-1, 256, 256, 1) 


### Step 2: creation of tensorflow dataset

Prompt 2:

Create a Tensorflow dataset from those arrays

In [27]:
import tensorflow as tf

# Assuming 'train' is your DataFrame with 'image' and 'label' columns
image_paths = list(train['image'])  # Extract image paths
mask_paths = list(train['label'])   # Extract mask paths

def create_ds(image_paths, mask_paths):
    # Load images and masks, ensuring they are the same size and normalized
    images = np.array([cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) for img_path in image_paths]) / 255.0
    images = images.reshape(-1, 256, 256, 1)
    
    labels = np.array([np.array(Image.open(mask_path).convert('L').resize((256, 256))) for mask_path in mask_paths]) / 255.0
    labels = labels.reshape(-1, 256, 256, 1) 
    # Create a TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((images, labels)).batch(4)
    return dataset

train = create_ds(image_paths, mask_paths)

test = create_ds(list(test['image']), list(test['label']))

## Fix after Step 3:
#### positioned above in the code

Prompt 4:

create_block() in create_unet() was not defined




In [28]:
def create_block(x, filters, mode):
    if mode == "down":
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        x = layers.MaxPooling2D()(x)
        x = layers.Dropout(0.1)(x)
    elif mode == "up":
        x = layers.Conv2DTranspose(filters, 3, strides=2, activation='relu', padding='same')(x)
        #x = layers.UpSampling2D()(x)     # <------ wrong!   // edited
        x = layers.Dropout(0.1)(x)
    return x

# Step 3: network creation

Prompt 3:

Now give me the code to create a neural network for image segmentation with this dataset using tensorflow


In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model

# Define the U-Net architecture
def create_unet(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # Encoding path (downsampling)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.1)(x)
    
    # Downsampling (repeat 4 times)
    for i in range(3):
        x = create_block(x, 32, 'down')
        
    # Bottleneck
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    
    for i in range(1): # <-------- iterator not included in answer, assuming 1 //edited
        x = layers.Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')(x)
        x = layers.Dropout(0.1)(x)
    
    # Upsampling (repeat 4 times)
    for i in range(3):
        x = create_block(x, 32, 'up')
        
    # Output layer
    outputs = layers.Conv2D(1, 1, activation='sigmoid', padding='same')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

## Step 4: training

Prompt 5:

Apply the u-net to the dataset

In [30]:
%%time
# Create a TensorFlow Dataset from the images and labels.
#testdataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(4)

# Normalize the input images (optional).
#dataset = dataset.map(lambda x, y: (x / 255., y / 255.))

#### ^^^ commented out manually

#Train the U-Net model with your data.
unet_model = create_unet((256, 256,1))     ## was (256,256) // edited
unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryIoU()])
history = unet_model.fit(train, epochs=50)

Epoch 1/50


2024-04-08 19:03:58.200370: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_16/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


44/44 [==============================] - 4s 18ms/step - loss: 0.6935 - binary_io_u_2: 0.2774
Epoch 2/50
44/44 [==============================] - 1s 18ms/step - loss: 0.6925 - binary_io_u_2: 0.2759
Epoch 3/50
44/44 [==============================] - 1s 18ms/step - loss: 0.6917 - binary_io_u_2: 0.2759
Epoch 4/50
44/44 [==============================] - 1s 18ms/step - loss: 0.6927 - binary_io_u_2: 0.3276
Epoch 5/50
44/44 [==============================] - 1s 18ms/step - loss: 0.6824 - binary_io_u_2: 0.2758
Epoch 6/50
44/44 [==============================] - 1s 18ms/step - loss: 0.6808 - binary_io_u_2: 0.2759
Epoch 7/50
44/44 [==============================] - 1s 18ms/step - loss: 0.6428 - binary_io_u_2: 0.2759
Epoch 8/50
44/44 [==============================] - 1s 18ms/step - loss: 0.5148 - binary_io_u_2: 0.5922
Epoch 9/50
44/44 [==============================] - 1s 18ms/step - loss: 0.4364 - binary_io_u_2: 0.7008
Epoch 10/50
44/44 [==============================] - 1s 18ms/step - loss: 0

### added manually 

In [31]:
loss, iou = unet_model.evaluate(test)
print(f"Test Loss: {loss}")
print(f"Test IoU: {iou}")

2/2 [==============================] - 0s 11ms/step - loss: 0.1174 - binary_io_u_2: 0.9050
Test Loss: 0.1174040213227272
Test IoU: 0.9050248861312866
